In [1]:
import json
import pandas as pd

In [2]:
with open("../source/StockFacts.json", 'r') as infile:
    data = infile.read()
    new_data = data.replace('}{', '},{')
    json_data = json.loads(f'[{new_data}]')

In [3]:
tickers = pd.read_csv("../source/compcom.csv")
tickers_df = tickers.drop(columns="url")

In [4]:
# # TEST RUN for revenue extraction
# for x in range(len(json_data)):
#     try:
#         #json_data[x]['data']['revenue']
#         print(x, json_data[x]['data']['revenue'])
#     except: 
#         print(x, "NaN")

In [5]:
# # TEST RUN for size extraction
# for x in range(len(json_data)):
#     try: 
#         print(x,json_data[x]['data']['size'])
#     except:
#         print(x,"NaN")

In [6]:
revenue = []
for x in range(len(json_data)):
    try:
        #json_data[x]['data']['revenue']
        revenue.append(json_data[x]['data']['revenue'])
        
    except: 
        revenue.append("NaN")

In [7]:
size = []
for x in range(len(json_data)):
    try:
        #json_data[x]['data']['revenue']
        size.append(json_data[x]['data']['size'])
        
    except: 
        size.append("NaN")

In [8]:
revenue_df = pd.DataFrame(revenue, columns = ['revenue'])

In [9]:
size_df = pd.DataFrame(size, columns = ['size'])

In [10]:
combine = [tickers_df, revenue_df, size_df]
size_revenue_df = pd.concat(combine, axis=1)
size_revenue_df.head()

# figured adding ticker column would make it easier to join on SQL?? yes? no?

,Ticker,revenue,size
0,AAPL,over-1b,over-10k
1,ABNB,200m-1b,5k-10k
2,ADBE,1m-10m,over-10k
3,ADI,over-1b,over-10k
4,ADP,over-1b,over-10k


In [11]:
# if need to change "none" and "NaN" to zero value
size_revenue_df = size_revenue_df.fillna(0)

In [12]:
size_revenue_df = size_revenue_df.replace("NaN", 0)

In [13]:
url_names_df = pd.read_csv("../source/compcom_edit.csv")

In [14]:
url_names_df = url_names_df.drop(columns="Ticker")

In [15]:
combine2 = [size_revenue_df, url_names_df]
size_revenue_df2 = pd.concat(combine2, axis=1)

In [16]:
# SECTOR NAMES & ABBREVIATIONS

# Healthcare             _H
# Materials              _M
# Real Estate            _R
# Consumer Staples       _CS
# Consumer Discretionary _CD
# Utilities              _U
# Energy                 _E
# Industrials            _I
# Communication Services _CM
# Financials             _F
# Technology             _T

In [17]:
sectors_df = pd.read_csv("../coordinates/SECTORS_edit.csv")

In [18]:
sectors_df.drop(["name","ticker"], axis=1, inplace=True)

In [19]:
sectors_df

,sector,sector_short
0,Technology,T
1,Technology,T
2,Technology,T
3,Technology,T
4,Consumer Discretionary,CD
...,...,...
97,Healthcare,H
98,Technology,T
99,Utilities,U
100,Technology,T


In [20]:
combine3 = [size_revenue_df2, sectors_df]
size_revenue_df3 = pd.concat(combine3, axis=1)
size_revenue_df3.rename(columns={"Ticker":"ticker"}, inplace=True)
size_revenue_df3.set_index("ticker", inplace=True)

In [21]:
size_revenue_df3

,revenue,size,url,sector,sector_short
ticker,,,,,
AAPL,over-1b,over-10k,apple.com,Technology,T
ABNB,200m-1b,5k-10k,airbnb.com,Technology,T
ADBE,1m-10m,over-10k,adobe.com,Technology,T
ADI,over-1b,over-10k,analog.com,Technology,T
ADP,over-1b,over-10k,adp.com,Consumer Discretionary,CD
...,...,...,...,...,...
WBA,0,over-10k,walgreensbootsalliance.com,Healthcare,H
WDAY,over-1b,over-10k,workday.com,Technology,T
XEL,0,over-10k,my.xcelenergy.com,Utilities,U


In [22]:
size_revenue_full = size_revenue_df3.drop("sector_short", axis=1)

In [23]:
#size_revenue_full

,revenue,size,url,sector
ticker,,,,
AAPL,over-1b,over-10k,apple.com,Technology
ABNB,200m-1b,5k-10k,airbnb.com,Technology
ADBE,1m-10m,over-10k,adobe.com,Technology
ADI,over-1b,over-10k,analog.com,Technology
ADP,over-1b,over-10k,adp.com,Consumer Discretionary
...,...,...,...,...
WBA,0,over-10k,walgreensbootsalliance.com,Healthcare
WDAY,over-1b,over-10k,workday.com,Technology
XEL,0,over-10k,my.xcelenergy.com,Utilities


In [24]:
#size_revenue_full.to_csv("../source/size_revenue_data.csv")

In [26]:
size_revenue_abb = size_revenue_df3.drop("sector",axis=1)
size_revenue_abb.rename(columns = {"sector_short":"sector"},inplace=True)

In [27]:
size_revenue_abb

,revenue,size,url,sector
ticker,,,,
AAPL,over-1b,over-10k,apple.com,T
ABNB,200m-1b,5k-10k,airbnb.com,T
ADBE,1m-10m,over-10k,adobe.com,T
ADI,over-1b,over-10k,analog.com,T
ADP,over-1b,over-10k,adp.com,CD
...,...,...,...,...
WBA,0,over-10k,walgreensbootsalliance.com,H
WDAY,over-1b,over-10k,workday.com,T
XEL,0,over-10k,my.xcelenergy.com,U


In [28]:
size_revenue_abb.to_csv("../source/size_revenue_data.csv")